Modified code originally provided by Zona.

In [ ]:
import torch
import torchvision.models as models
from torchvision import transforms as trn
from torch.nn import functional as F
import pickle
import numpy as np
from PIL import Image
import cv2
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image, \
    deprocess_image, \
    preprocess_image

In [ ]:
def get_cam(img):
  '''
  returns input image with CAM heatmap overlay
  '''

  #load pretrained places CNN model provided by Zona
  with open('cnn_model.pkl', 'rb') as f:
    cnn_model = pickle.load(f)

  # process image before being input into CNN model
  centre_crop = trn.Compose([
          trn.Resize((256,256)),
          trn.CenterCrop(224),
          trn.ToTensor(),
          trn.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
  ])

  with torch.no_grad():
    input_img = centre_crop(img).unsqueeze(0)

  # forward pass to classify image
  logit = cnn_model.forward(input_img)
  h_x = F.softmax(logit, 1).data.squeeze()
  probs, idx = h_x.sort(0, True)

  #process input image before overlay CAM
  img = np.array(img)
  img = cv2.resize(img, (224, 224))
  img = np.float32(img) / 255
  input_tensor = preprocess_image(img, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

  #overlay CAM on image
  targets = [ClassifierOutputTarget(idx[0])]
  target_layers = [cnn_model.layer4]
  with GradCAM(model=cnn_model, target_layers=target_layers) as cam:
      grayscale_cams = cam(input_tensor=input_tensor, targets=targets)
      cam_image = show_cam_on_image(img, grayscale_cams[0, :], use_rgb=True)

  return cam_image